In [1]:
%load_ext autoreload
%autoreload 2

### giant otter vocalizations
- ~500 vocalizations from https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0112562

In [2]:
context_dict = {
    0:'Unknown',
    1:'Separation',
    2:'Biting',
    3:'Feeding',
    4:'Fighting',
    5:'Grooming',
    6:'Isolation',
    7:'Kissing',
    8:'Landing',
    9:'Mating protest',
    10:'Threat-like',
    11:'General',
    12:'Sleeping',
}

In [3]:
from joblib import Parallel, delayed
from tqdm.autonotebook import tqdm
import pandas as pd
import librosa
from datetime import datetime
import json
from IPython.display import display

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
from avgn.utils.paths import DATA_DIR, ensure_dir

In [6]:
DSLOC = DATA_DIR / "raw/batsong/"

In [7]:
annotations =  pd.read_csv(DSLOC/'Annotations.csv')
len(annotations)

91080

In [8]:
annotations[:3]

,FileID,Emitter,Addressee,Context,Emitter pre-vocalization action,Addressee pre-vocalization action,Emitter post-vocalization action,Addressee post-vocalization action,Start sample,End sample
0,7,118,0,9,2,2,3,3,1,336720
1,11,0,0,11,0,0,0,0,1,787280
2,12,118,0,12,2,2,3,3,1,566096


In [9]:
file_info = pd.read_csv(DSLOC / 'FileInfo.csv', names = np.arange(50))
file_info.columns = file_info.loc[0].values
file_info = file_info[1:]
file_info = file_info.set_index('FileID')
file_info.index = file_info.index.astype('int')
len(file_info)

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (0,1,4,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


293238

In [10]:
file_info[:3]

,Treatment ID,File name,File folder,Recording channel,Recording time,Voice start sample (1),Voice end sample (1),Voice start sample (2),Voice end sample (2),...,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
FileID,,,,,,,,,,,,,,,,,,,,,
1,1,120601000005102988.WAV,files101,11,2012-06-01 00:01:57,52100,120902,130577,158906,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,120601000005108269.WAV,files101,4,2012-06-01 00:01:57,28895,89442,102612,129387,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,120601000011795989.WAV,files101,11,2012-06-01 00:02:04,54143,100989,101607,121337,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
DSLOC

PosixPath('/mnt/cube/tsainbur/Projects/github_repos/avgn_paper/data/raw/batsong')

In [12]:
wavfiles = list(DSLOC.glob('zip_contents/*/*.WAV'))
len(wavfiles), wavfiles[0]

(263436,
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/avgn_paper/data/raw/batsong/zip_contents/files102/120627013338667299.WAV'))

In [13]:
wf_df = pd.DataFrame(
    [[wf, wf.name, wf.parent.stem] for wf in tqdm(wavfiles)],
    columns=["wav_loc", "id", "folder"],
)
wf_df = wf_df.set_index('id')
len(wf_df)

263436

In [14]:
wf_df[:3]

,wav_loc,folder
id,,
120627013338667299.WAV,/mnt/cube/tsainbur/Projects/github_repos/avgn_...,files102
120702045930363917.WAV,/mnt/cube/tsainbur/Projects/github_repos/avgn_...,files102
120621233540868238.WAV,/mnt/cube/tsainbur/Projects/github_repos/avgn_...,files102


### create json

In [15]:
sr = 250000

In [16]:
from avgn.utils.audio import get_samplerate
from avgn.utils.json import NoIndent, NoIndentEncoder
from datetime import datetime
import librosa
import json

In [17]:
DATASET_ID = "batsong"

In [18]:
# create a unique datetime identifier for the files output by this notebook
DT_ID = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
DT_ID

'2019-10-01_19-28-42'

In [19]:
display(annotations[:1])
display(wf_df[:1])
display(file_info[:1])

,FileID,Emitter,Addressee,Context,Emitter pre-vocalization action,Addressee pre-vocalization action,Emitter post-vocalization action,Addressee post-vocalization action,Start sample,End sample
0,7,118,0,9,2,2,3,3,1,336720


,wav_loc,folder
id,,
120627013338667299.WAV,/mnt/cube/tsainbur/Projects/github_repos/avgn_...,files102


,Treatment ID,File name,File folder,Recording channel,Recording time,Voice start sample (1),Voice end sample (1),Voice start sample (2),Voice end sample (2),...,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
FileID,,,,,,,,,,,,,,,,,,,,,
1,1,120601000005102988.WAV,files101,11,2012-06-01 00:01:57,52100,120902,130577,158906,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
len(annotations.FileID.unique())

87986

In [21]:
from avgn.utils.json import NoIndent, NoIndentEncoder

In [22]:
missing_wav = 0
# for each unique wav
for FID in tqdm(annotations.FileID.unique()):
    file_row = file_info.loc[FID]
    try:
        wav_row = wf_df.loc[file_row["File name"]]
    except KeyError as e:
        missing_wav += 1
        continue
    
    # get the file
    file_annotations = annotations[annotations.FileID == FID].sort_values(
        by="Start sample"
    )
            
    json_out = (
        DATA_DIR
        / "processed"
        / DATASET_ID
        / DT_ID
        / "JSON"
        / (wav_row.wav_loc.stem.replace(" ", "_") + ".JSON")
    )

    rate = get_samplerate(wav_row.wav_loc.as_posix())
    bout_duration = librosa.get_duration(filename=wav_row.wav_loc.as_posix())

    # wav general information
    json_dict = {}
    json_dict["fid"] = int(file_row.name)
    json_dict["Treatment ID"] = file_row["Treatment ID"]
    json_dict["Recording channel"] = file_row["Recording channel"]
    json_dict["Recording time"] = file_row["Recording time"]

    json_dict["indvs"] = {}
    for emitter in file_annotations.Emitter.unique():
        emitter_anno = file_annotations[file_annotations.Emitter == emitter]

        json_dict["indvs"][str(emitter)] = {
            "syllables": {
                "start_times": NoIndent(
                    list(emitter_anno["Start sample"].astype("float32") / sr)
                ),
                "end_times": NoIndent(
                    list(emitter_anno["End sample"].astype("float32") / sr)
                ),
                "addressee": NoIndent(list(emitter_anno["Addressee"].astype("int"))),
                "context": NoIndent([context_dict[i] for i in emitter_anno["Context"]]),
                "emit_prevoc_act": NoIndent(
                    list(emitter_anno["Emitter pre-vocalization action"].astype("int"))
                ),
                "add_prevoc_act": NoIndent(
                    list(
                        emitter_anno["Addressee pre-vocalization action"].astype("int")
                    )
                ),
                "emit_postvoc_act": NoIndent(
                    list(emitter_anno["Emitter post-vocalization action"].astype("int"))
                ),
                "add_postvoc_act": NoIndent(
                    list(
                        emitter_anno["Addressee post-vocalization action"].astype("int")
                    )
                ),
            }
        }

    json_dict["species"] = "Rousettus aegyptiacus"
    json_dict["common_name"] = "Egyptian fruit bat"
    json_dict["samplerate_hz"] = rate
    json_dict["original_wav"] = wav_row.wav_loc.as_posix()
    json_dict["length_s"] = bout_duration
    json_dict["wav_loc"] = wav_row.wav_loc.as_posix()

    json_txt = json.dumps(json_dict, cls=NoIndentEncoder, indent=2)

    # save json
    ensure_dir(json_out.as_posix())

    print(json_txt, file=open(json_out.as_posix(), "w"))

    #break

In [23]:
print(json_txt)

{
  "fid": 273433,
  "Treatment ID": 19,
  "Recording channel": 2,
  "Recording time": "2013-06-04 02:09:14",
  "indvs": {
    "-211": {
      "syllables": {
        "start_times": [4e-06],
        "end_times": [1.281344],
        "addressee": [-208],
        "context": ["Feeding"],
        "emit_prevoc_act": [2],
        "add_prevoc_act": [2],
        "emit_postvoc_act": [3],
        "add_postvoc_act": [3]
      }
    }
  },
  "species": "Rousettus aegyptiacus",
  "common_name": "Egyptian fruit bat",
  "samplerate_hz": 250000,
  "original_wav": "/mnt/cube/tsainbur/Projects/github_repos/avgn_paper/data/raw/batsong/zip_contents/files223/130604011439064237.WAV",
  "length_s": 1.281344,
  "wav_loc": "/mnt/cube/tsainbur/Projects/github_repos/avgn_paper/data/raw/batsong/zip_contents/files223/130604011439064237.WAV"
}
